In [1]:
import numpy as np
import os
import sys 
sys.path.append("../..")
import torch
import torch.nn as nn
import torch.utils.data as Data
from torch.nn import functional as F
import utils.transforms as trans
import utils.utils as util
import layer.loss as ls
import utils.metric as mc
import shutil
import cv2

import cfg.CDD as cfg
import dataset.rs as dates
import time
import datetime
from datetime import datetime
import logging

In [32]:
thres = np.linspace(0, 1, 5)   # array([0.  , 0.25, 0.5 , 0.75, 1.  ])
gtBin = np.array([[True, False, False],
                 [True, True, True],
                 [False, False, True]])

cur_prob = np.array([[0.7, 0.2, 0.4],
                 [0.6, 0.8, 0.5],
                 [0.3, 0.5, 0.9]])

thresInf = np.concatenate(([-np.Inf], thres, [np.Inf]))   # array([-inf, 0.  , 0.25, 0.5 , 0.75, 1.  ,  inf])

fnArray = cur_prob[(gtBin == True)]   # array([0.7, 0.6, 0.8, 0.5, 0.9])
fnHist = np.histogram(fnArray, bins=thresInf)[0]  # array([0, 0, 0, 3, 2, 0], dtype=int64)
fnCum = np.cumsum(fnHist)  # array([0, 0, 0, 3, 5, 5], dtype=int64)
FN = fnCum[0:0 + len(thres)]   # array([0, 0, 0, 3, 5], dtype=int64)

fpArray = cur_prob[(gtBin == False)]  # array([0.2, 0.4, 0.3, 0.5])
fpHist = np.histogram(fpArray, bins=thresInf)[0]  # array([0, 1, 2, 1, 0, 0], dtype=int64)
# 倒置求累计和再倒置回去
fpCum = np.flipud(np.cumsum(np.flipud(fpHist))) # array([4, 4, 3, 1, 0, 0], dtype=int64)
FP = fpCum[1:1 + len(thres)]  # array([4, 3, 1, 0, 0], dtype=int64)

posNum = np.sum(gtBin == True)
negNum = np.sum(gtBin == False)